In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv('C:\\Users\\Pablo\\MachineLearningTest\\data\\train.csv')

In [3]:
train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


Cambiar valores de la columna "num"

In [4]:
for i in range(len(train.num)):
    if train.num[i]!=0:
        train.loc[i,'num']=1

In [5]:
train = train.fillna(0)
X, y = train.iloc[:, :-1], train.iloc[:, -1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
y_pred = log.fit(X_train,y_train).predict(X_test)

In [6]:
##MATRIZ DE CONFUSION DEL MODELO
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[48,  4],
       [10, 26]], dtype=int64)

In [7]:
##RESULTADO DE LA PREDICCION
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.84090909090909094

Hacemos 9 componentes principales para mejorar el modelo

In [8]:
from sklearn.decomposition import PCA
pca = PCA(9)
Xtr =pca.fit(X).transform(X)
X_train, X_test, y_train, y_test = train_test_split(Xtr, y, test_size=0.33)
log = LogisticRegression()
y_pred = log.fit(X_train,y_train).predict(X_test)

In [9]:
confusion_matrix(y_test, y_pred)

array([[40,  4],
       [ 8, 36]], dtype=int64)

In [10]:
accuracy_score(y_test, y_pred)

0.86363636363636365

In [38]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def traintest(c, k, pert_split=0.33):
    #result=[]
   
    pca = PCA(k)   
    Xtr =pca.fit(X).transform(X)
        
    X_train, X_test, y_train, y_test = train_test_split(Xtr, y, test_size=0.33)
    result_int=[]
        
        #result_int.append(X_train)
        #result_int.append(y_train)
        #result_int.append(X_test)
        #result_int.append(y_test)
        
    log = LogisticRegression(C=c)
    y_pred = log.fit(X_train,y_train).predict(X_test)
        
        #result_int.append(y_pred)
        
    mat_conf=confusion_matrix(y_test, y_pred)
        
    result_int.append(mat_conf)
        
    scor=accuracy_score(y_test, y_pred)
       
    result_int.append(scor)
        
    #result.append(result_int)
    
    return result_int

In [42]:
res= traintest(0.1, 9)
res[0]

array([[43, 10],
       [ 8, 27]], dtype=int64)

In [13]:
import itertools
def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

In [50]:
c= np.arange(0.1, 1, 0.05)
k= np.arange(6, 11, 1)
expandgrid(c, k)

parametres = pd.DataFrame.from_dict(expandgrid(c, k))
parametres.columns = ['C','k']

In [51]:
TP = []
TN = []
FP = []
FN = []
AC = []
for i in range(parametres.shape[0]):
    result = traintest(parametres.loc[i,'C'],parametres.loc[i,'k'])
    confmat = result[0]
    TP.append(confmat[1, 1])
    TN.append(confmat[0, 0])
    FP.append(confmat[0, 1])
    FN.append(confmat[1, 0])
    AC.append(result[1])
parametres["TP"]=TP
parametres["TN"]=TN
parametres["FP"]=FP
parametres["FN"]=FN
parametres["AC"]=AC

In [52]:
parametres

,C,k,TP,TN,FP,FN,AC
0,0.10,6,28,39,10,11,0.761364
1,0.10,7,27,41,11,9,0.772727
2,0.10,8,31,48,3,6,0.897727
3,0.10,9,27,46,4,11,0.829545
4,0.10,10,32,45,5,6,0.875000
5,0.15,6,27,46,5,10,0.829545
6,0.15,7,28,47,8,5,0.852273
7,0.15,8,29,44,10,5,0.829545
8,0.15,9,31,42,6,9,0.829545
9,0.15,10,28,42,10,8,0.795455


In [56]:
df.plot(x='TP', y='FN')

NameError: name 'df' is not defined